In [2]:
from tqdm.auto import tqdm
import torch
import json
import os
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from dataloader import * 

In [3]:
data_list = torch.load('data/training/data_list_0.pt')

In [17]:
torch.stack((torch.tensor(data_list[0][1][0]), torch.tensor(data_list[4][1][0])), dim=0)

tensor([34.8000, 30.7000], dtype=torch.float64)

In [12]:
torch.tensor(data_list[0][1])

tensor([34.8000, 36.5000], dtype=torch.float64)

In [3]:
running_prod = 1
for i in range(100):
    running_prod *= (1 - (2*(1 / ((52**2)-(2*i)))))
print(1-running_prod)

0.07396449704142083


In [11]:
with open('hands_valid.json', 'r') as f_in:
    with open('2p_hands.json', 'w') as f_out:
        for line in tqdm(f_in):
            data = json.loads(line)
            if(data['num_players'] == 2):
                f_out.write(line)

402126it [00:05, 76640.57it/s]


In [60]:
maximum_size = 0
with open('super_filtered.json', 'r') as file:
    i = 0
    for line in tqdm(file):
        data = json.loads(line)
        action_count = 0
        for round in data['players'][0]['bets']:
            action_count += len(round['actions'])
        for round in data['players'][1]['bets']:
            action_count += len(round['actions'])
        if(action_count > maximum_size):
            maximum_size = action_count
print(maximum_size)

367055it [00:03, 95409.81it/s]

20


In [57]:
with open('hands_valid.json', 'r') as f_in:
    with open('super_filtered.json', 'w') as f_out:
        for line in tqdm(f_in):
            data = json.loads(line)
            if(data['num_players'] == 2):
                action_count = 0
                for round in data['players'][0]['bets']:
                    action_count += len(round['actions'])
                for round in data['players'][1]['bets']:
                    action_count += len(round['actions'])
                if(action_count <= 20):
                    f_out.write(line)

402126it [00:06, 66596.71it/s]


In [59]:
with open('2p_hands.json', 'r') as f_in:
    for lin in tqdm(f_in):
        None

369772it [00:00, 723615.60it/s]


In [24]:
def encode_bets(bets):
    '''
    One-hot encodes the rounds of the game (pre-flop, flop, turn, and river) as well as the many
    actions a SINGLE player has made. i.e., we need to call this once per player
    '''
    round_encoder = OneHotEncoder(categories=[['p', 'f', 't', 'r']])
    action_encoder = OneHotEncoder(categories=[['-', 'B', 'f', 'k', 'b', 'c', 'r', 'A', 'Q', 'K']])
    round_list, action_list = [], []
    for round in bets:
        for action in round['actions']:
            round_list.append(round['stage'])
            action_list.append(action)
    rounds = round_encoder.fit_transform(np.array(round_list).reshape(-1, 1)).toarray()
    actions = action_encoder.fit_transform(np.array(action_list).reshape(-1,1)).toarray()
    return torch.tensor(rounds.T), torch.tensor(actions)

In [64]:
round_encoder = OneHotEncoder(categories=[['p', 'f', 't', 'r']])
action_encoder = OneHotEncoder(categories=[['-', 'B', 'f', 'k', 'b', 'c', 'r', 'A', 'Q', 'K']])

full_data_list = []

with open('super_filtered.json', 'r') as file:
    i = 0
    for line in tqdm(file):
        if (i > 0): break
        data = json.loads(line)
        player_1 = data['players'][0]['bets']
        player_2 = data['players'][1]['bets']
        mixed_action_list = []
        player_list = []
        round_list = []
        for p1_round, p2_round in zip(player_1, player_2):
            j = 0
            while(True):
                try:
                    mixed_action_list.append(p1_round['actions'][j])
                    player_list.append(0)
                    round_list.append(p1_round['stage'])
                except:
                    break
                try:
                    mixed_action_list.append(p2_round['actions'][j])
                    player_list.append(1)
                    round_list.append(p2_round['stage'])
                except:
                    break
                j += 1
        mixed_action_tensor = torch.tensor(action_encoder.fit_transform(np.array(mixed_action_list).reshape(-1,1)).toarray().T)
        round_tensor = torch.tensor(round_encoder.fit_transform(np.array(round_list).reshape(-1,1)).toarray().T)
        player_tensor = torch.tensor(player_list).reshape(1, len(player_list))
        full_tensor = torch.concat((round_tensor, player_tensor, mixed_action_tensor))
        padded_tensor = torch.nn.functional.pad(full_tensor, (0, maximum_size-full_tensor.shape[1]), mode='constant', value=0)
        full_data_list.append(padded_tensor)
torch.save(full_data_list, 'huge_list.pt')

367055it [03:25, 1785.48it/s]


torch.Size([15, 20])

In [71]:
random_indices = np.random.permutation(range(len(full_data_list)))
batch_size = 64
batches = []
for i in tqdm(range(0, len(full_data_list), batch_size)):
    end_index = min((i+batch_size), len(full_data_list))
    tensor_list = [full_data_list[random_indices[j]] for j in range(i, end_index)]
    batch = torch.stack(tensor_list, dim=0)
    batches.append(batch)

100%|██████████| 5736/5736 [00:00<00:00, 5915.79it/s]


In [3]:
from dataloader import load_batches
train, val, test = load_batches(batch_size=64)

Shuffling data (outside of batches!)


100%|██████████| 574/574 [00:00<00:00, 1848.64it/s]


In [4]:
train[0].shape

torch.Size([64, 20, 15])

In [12]:
inputs = torch.argmax(torch.randn(128, 10), dim=1)
targets = torch.argmax(torch.randn(128, 10), dim=1)
torch.count_nonzero(inputs == targets)/inputs.shape[0]

tensor(0.0781)

In [87]:
train[0][:, -10:, 1:].shape

torch.Size([64, 10, 19])

In [88]:
train[0][:,:,:-1].shape

torch.Size([64, 15, 19])

In [30]:
def my_loss(output, target):
    loss = torch.mean((output - target)**2)
    return loss

model = torch.nn.Linear(2, 2)
x = torch.randn(1, 2)
target = torch.randn(1, 2)
output = model(x)
loss = my_loss(output, target)
loss.backward()
print(loss.item())

2.553941249847412


In [19]:
torch.load('training/input_data_001.pt')[0].shape

torch.Size([8, 45])

In [4]:
torch.load('training/target_data_001.pt')[0].shape

torch.Size([1, 104])

In [5]:
t = torch.load('training/target_data_001.pt')[0]

In [6]:
y = torch.load('training/target_data_001.pt')[1]

In [15]:
yc1, yc2 = torch.split(y, 52, dim=1)

In [14]:
tc1, tc2 = torch.split(t, 52, dim=1)

In [ ]:
yc1

In [18]:
criterion = torch.nn.CrossEntropyLoss()
torch.add(criterion(yc1, tc1), criterion(yc2, tc2))

tensor(7.9675, dtype=torch.float64)

In [ ]:
def fourLoss(y, t):
    yc1, yc2 = torch.split(y, 52, dim=0)
    tc1, tc2 = torch.split(t, 52, dim=0)